# Chapter 04-03: TFRecord 포맷

## 학습 목표
- TFRecord 형식의 구조와 대용량 데이터에서의 이점을 이해한다
- `tf.train.Example` / `tf.train.Feature`로 데이터를 직렬화한다
- TFRecord 파일을 쓰고 읽는 전체 워크플로우를 구현한다
- 파싱 함수와 `tf.data` 파이프라인을 연결한다

## 목차
1. TFRecord가 필요한 이유
2. tf.train.Feature 3가지 타입
3. MNIST → TFRecord 변환
4. TFRecord 읽기

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import time

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

print('TensorFlow 버전:', tf.__version__)

# MNIST 데이터 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(f'MNIST 훈련: {x_train.shape}, 테스트: {x_test.shape}')

# TFRecord 저장 경로 설정
TFRECORD_DIR = pathlib.Path('/tmp/mnist_tfrecords')
TFRECORD_DIR.mkdir(parents=True, exist_ok=True)
TRAIN_TFR = str(TFRECORD_DIR / 'train.tfrecord')
TEST_TFR  = str(TFRECORD_DIR / 'test.tfrecord')
print(f'TFRecord 저장 경로: {TFRECORD_DIR}')

## TFRecord가 필요한 이유

**일반 이미지 파일**: 각 파일을 개별로 열어야 → 대용량에서 I/O 병목

**TFRecord**: 모든 데이터를 하나의 이진 파일로 → 순차 읽기 최적화

### 파일 형식 비교

| 방식 | 파일 수 | 읽기 방식 | 대용량 성능 |
|------|---------|-----------|------------|
| 개별 JPEG/PNG | 데이터 수만큼 | 랜덤 접근 (비효율) | 느림 |
| TFRecord | 몇 개의 큰 파일 | 순차 읽기 (최적화) | 빠름 |

### TFRecord 파일 구조
```
TFRecord 파일
├── Record 1: tf.train.Example (직렬화된 protobuf)
│     └── tf.train.Features
│           ├── 'image': bytes_list
│           └── 'label': int64_list
├── Record 2: tf.train.Example
│     └── ...
└── Record N: tf.train.Example
```

## tf.train.Feature 3가지 타입

In [ ]:
# -----------------------------------------------------------
# tf.train.Feature는 세 가지 타입만 지원한다:
#   1. bytes_list  — 문자열, 바이트 배열, 직렬화된 이미지
#   2. float_list  — 부동소수점 수 (float32)
#   3. int64_list  — 정수 (int64)
# -----------------------------------------------------------

# 1. bytes_list: 이미지 raw bytes나 문자열 저장에 사용
def bytes_feature(value):
    """bytes 또는 str을 bytes_list Feature로 변환한다."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # EagerTensor → bytes 변환
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 2. float_list: 정규화된 픽셀값, 임베딩 벡터 등
def float_feature(value):
    """float 또는 float 리스트를 float_list Feature로 변환한다."""
    if not isinstance(value, (list, np.ndarray)):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

# 3. int64_list: 레이블, 인덱스, 정수형 메타데이터
def int64_feature(value):
    """int 또는 int 리스트를 int64_list Feature로 변환한다."""
    if not isinstance(value, (list, np.ndarray)):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

# 각 타입 동작 확인
ex_bytes = bytes_feature(b'hello tfrecord')
ex_float = float_feature([0.1, 0.2, 0.9])
ex_int   = int64_feature([3, 7, 42])

print('bytes_list Feature:')
print(ex_bytes)
print('float_list Feature:')
print(ex_float)
print('int64_list Feature:')
print(ex_int)

## MNIST → TFRecord 변환

In [ ]:
# -----------------------------------------------------------
# create_tf_example: 이미지 + 레이블 → tf.train.Example
#
# 이미지는 bytes_list에 저장한다:
#   - numpy ndarray.tobytes()로 raw bytes 직렬화
#   - shape 정보도 별도 Feature로 저장해야 복원 가능
# -----------------------------------------------------------

def create_tf_example(image, label):
    """
    image: numpy uint8 배열 (28, 28)
    label: int
    returns: 직렬화된 tf.train.Example bytes
    """
    # 이미지를 raw bytes로 변환
    image_bytes = image.tobytes()

    feature = {
        'image/raw':    bytes_feature(image_bytes),      # 이미지 raw bytes
        'image/height': int64_feature(image.shape[0]),   # 높이 (28)
        'image/width':  int64_feature(image.shape[1]),   # 너비 (28)
        'label':        int64_feature(int(label)),       # 레이블 (0-9)
    }

    tf_example = tf.train.Example(
        features=tf.train.Features(feature=feature)
    )
    # SerializeToString(): protobuf → bytes (파일에 쓸 수 있는 형태)
    return tf_example.SerializeToString()

# 단일 샘플 테스트
test_example = create_tf_example(x_train[0], y_train[0])
print(f'직렬화된 Example 크기: {len(test_example)} bytes')
print(f'원본 이미지 크기: {x_train[0].nbytes} bytes')
print(f'오버헤드: {len(test_example) - x_train[0].nbytes} bytes (Feature 키 + protobuf 헤더)')

In [ ]:
# -----------------------------------------------------------
# TFRecord 파일 쓰기
# tf.io.TFRecordWriter를 컨텍스트 매니저로 사용하면
# 파일이 자동으로 닫히고 flush된다
# -----------------------------------------------------------

def write_tfrecord(images, labels, filepath, max_samples=None):
    """
    images, labels 배열을 TFRecord 파일로 저장한다.
    max_samples: 저장할 최대 샘플 수 (None이면 전체)
    """
    if max_samples is not None:
        images = images[:max_samples]
        labels = labels[:max_samples]

    start = time.time()
    with tf.io.TFRecordWriter(filepath) as writer:
        for i, (img, lbl) in enumerate(zip(images, labels)):
            serialized = create_tf_example(img, lbl)
            writer.write(serialized)
            if (i + 1) % 5000 == 0:
                print(f'  {i+1}/{len(images)} 기록 완료...')

    elapsed = time.time() - start
    file_size = os.path.getsize(filepath) / 1024 / 1024
    print(f'저장 완료: {filepath}')
    print(f'  샘플 수: {len(images)}, 파일 크기: {file_size:.2f} MB, 소요 시간: {elapsed:.1f}초')

# 훈련 데이터 전체 → TFRecord (처음 실행 시 약 10-20초 소요)
print('=== 훈련 TFRecord 생성 ===')
write_tfrecord(x_train, y_train, TRAIN_TFR)

print('\n=== 테스트 TFRecord 생성 ===')
write_tfrecord(x_test, y_test, TEST_TFR)

## TFRecord 읽기

In [ ]:
# -----------------------------------------------------------
# 파싱 함수: 직렬화된 bytes → (이미지 텐서, 레이블 텐서)
#
# tf.io.parse_single_example():
#   - serialized: 단일 직렬화 Example
#   - features: Feature 이름 → 예상 타입/형태 명세
# -----------------------------------------------------------

# Feature 명세 (쓸 때와 동일한 키/타입 사용)
FEATURE_DESCRIPTION = {
    'image/raw':    tf.io.FixedLenFeature([], tf.string),   # bytes_list
    'image/height': tf.io.FixedLenFeature([], tf.int64),    # int64_list
    'image/width':  tf.io.FixedLenFeature([], tf.int64),    # int64_list
    'label':        tf.io.FixedLenFeature([], tf.int64),    # int64_list
}

def parse_fn(serialized_example):
    """
    직렬화된 tf.train.Example을 파싱하여
    (이미지 텐서, 레이블 텐서) 쌍을 반환한다.
    """
    # protobuf 파싱
    parsed = tf.io.parse_single_example(serialized_example, FEATURE_DESCRIPTION)

    # raw bytes → uint8 텐서 복원
    image = tf.io.decode_raw(parsed['image/raw'], tf.uint8)

    # flat 텐서 → 원래 shape으로 reshape
    h = parsed['image/height']
    w = parsed['image/width']
    image = tf.reshape(image, [h, w])

    # 정규화: [0,255] → [0.0, 1.0]
    image = tf.cast(image, tf.float32) / 255.0

    # 채널 차원 추가: (28,28) → (28,28,1)
    image = tf.expand_dims(image, axis=-1)

    label = parsed['label']
    return image, label

# 파싱 함수 단독 테스트
raw_ds = tf.data.TFRecordDataset(TRAIN_TFR)
for raw_record in raw_ds.take(1):
    img, lbl = parse_fn(raw_record)
    print(f'파싱 후 이미지 shape: {img.shape}, dtype: {img.dtype}')
    print(f'파싱 후 레이블: {lbl.numpy()}')

In [ ]:
# -----------------------------------------------------------
# TFRecord → tf.data 파이프라인 구성
# -----------------------------------------------------------

AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64

def load_tfrecord_dataset(filepath, training=True, batch_size=BATCH_SIZE):
    """
    TFRecord 파일에서 완전한 학습용 Dataset을 생성한다.
    num_parallel_reads: 여러 TFRecord 파일을 병렬로 읽을 때 사용
    """
    ds = tf.data.TFRecordDataset(
        filepath,
        num_parallel_reads=AUTOTUNE  # 파일이 여러 개일 때 병렬 읽기
    )
    # 파싱을 병렬로 수행 (CPU 코어 활용)
    ds = ds.map(parse_fn, num_parallel_calls=AUTOTUNE)
    ds = ds.cache()                           # 파싱 결과를 메모리에 캐시
    if training:
        ds = ds.shuffle(buffer_size=10_000)
    ds = ds.batch(batch_size, drop_remainder=training)
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = load_tfrecord_dataset(TRAIN_TFR, training=True)
test_ds  = load_tfrecord_dataset(TEST_TFR,  training=False)

print('훈련 Dataset spec:', train_ds.element_spec)
print('테스트 Dataset spec:', test_ds.element_spec)

# 배치 수 확인
n_train_batches = sum(1 for _ in train_ds)
n_test_batches  = sum(1 for _ in test_ds)
print(f'\n훈련 배치 수: {n_train_batches}  (60000 // 64 = {60000 // 64})')
print(f'테스트 배치 수: {n_test_batches}  (10000 // 64 ≈ {10000 // 64})')

In [ ]:
# -----------------------------------------------------------
# TFRecord 파이프라인에서 로드된 데이터 시각화
# -----------------------------------------------------------

for imgs, labels in train_ds.take(1):
    fig, axes = plt.subplots(3, 8, figsize=(16, 6))
    fig.suptitle('TFRecord에서 로드된 MNIST 이미지 (배치 크기=64 중 24개)', fontsize=13)

    for i, ax in enumerate(axes.flat):
        ax.imshow(imgs[i].numpy().squeeze(), cmap='gray')
        ax.set_title(f'레이블: {labels[i].numpy()}', fontsize=8)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

print(f'배치 이미지 shape: {imgs.shape}  — (64, 28, 28, 1)')
print(f'픽셀값 범위: [{imgs.numpy().min():.3f}, {imgs.numpy().max():.3f}]')

In [ ]:
# -----------------------------------------------------------
# (심화) 다중 TFRecord 파일 분할
#
# 대규모 데이터셋에서는 하나의 TFRecord 대신
# 여러 개의 shard 파일로 분할하면:
#   1. 병렬 읽기 가능 (num_parallel_reads=AUTOTUNE)
#   2. 파일별로 캐시/스트리밍 관리 용이
# -----------------------------------------------------------

NUM_SHARDS = 4  # 4개 파일로 분할
SHARD_DIR = TFRECORD_DIR / 'shards'
SHARD_DIR.mkdir(exist_ok=True)

# 샘플 수 / 샤드 수 = 샤드당 샘플 수
shard_size = len(x_train) // NUM_SHARDS

for shard_id in range(NUM_SHARDS):
    shard_path = str(SHARD_DIR / f'train-{shard_id:05d}-of-{NUM_SHARDS:05d}.tfrecord')
    start_idx = shard_id * shard_size
    end_idx   = start_idx + shard_size

    with tf.io.TFRecordWriter(shard_path) as writer:
        for img, lbl in zip(x_train[start_idx:end_idx], y_train[start_idx:end_idx]):
            writer.write(create_tf_example(img, lbl))

    print(f'  shard {shard_id+1}/{NUM_SHARDS}: {shard_path} ({shard_size}개)')

# glob 패턴으로 모든 shard 로드
shard_files = sorted(SHARD_DIR.glob('*.tfrecord'))
shard_ds = tf.data.TFRecordDataset(
    [str(f) for f in shard_files],
    num_parallel_reads=AUTOTUNE  # 4개 파일을 병렬로 읽음
).map(parse_fn, num_parallel_calls=AUTOTUNE)

print(f'\n{NUM_SHARDS}개 shard에서 로드된 총 샘플 수: {sum(1 for _ in shard_ds)}')

## 정리

### TFRecord 워크플로우 요약

```
[쓰기]
numpy 배열
  └→ bytes_feature / int64_feature / float_feature
       └→ tf.train.Features
            └→ tf.train.Example
                 └→ .SerializeToString()
                      └→ TFRecordWriter.write()

[읽기]
TFRecord 파일
  └→ tf.data.TFRecordDataset
       └→ .map(parse_fn)
            └→ tf.io.parse_single_example()
                 └→ tf.io.decode_raw() / decode_jpeg() 등
                      └→ reshape → normalize
                           └→ .cache().shuffle().batch().prefetch()
```

| 항목 | 설명 |
|------|------|
| `bytes_list` | 이미지 raw bytes, 문자열, 직렬화된 텐서 |
| `float_list` | 부동소수점 수, 정규화된 픽셀, 임베딩 |
| `int64_list` | 레이블, 정수 메타데이터, shape 정보 |
| `FixedLenFeature` | 길이가 고정된 Feature 파싱 |
| `VarLenFeature` | 길이가 가변적인 Feature 파싱 |

**다음**: practice/ex01_build_data_pipeline.ipynb — 종합 실습